In [1]:
import os

import pandas as pd
import numpy as np

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine

from flask import Flask, jsonify, render_template
from flask_sqlalchemy import SQLAlchemy

app = Flask(__name__)

In [2]:
#################################################
# Database Setup
#################################################

app.config["SQLALCHEMY_DATABASE_URI"] = "sqlite:///db/db_congress.db"
db = SQLAlchemy(app)

# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(db.engine, reflect=True)

# Save references to each table
Members_Metadata = Base.classes.members



c:\users\adam\anaconda\anaconda3\envs\pythondata\lib\site-packages\flask_sqlalchemy\__init__.py:835: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  'SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and '


In [57]:
def line(state):
    stmt = db.session.query(Members_Metadata).statement
    df = pd.read_sql_query(stmt, db.session.bind)
    
    # Filter the data for the states selected
#     df_filtered = pd.DataFrame()
    
    df_filtered = df.loc[df['State'] == state].reset_index()
    
    # Group the dataframes
    df_filtered_grouped = df_filtered['SwornInAge'].groupby(df_filtered['Year'])
    df_grouped = df['SwornInAge'].groupby(df['Year'])
    df_ByState = df['SwornInAge'].groupby([df['State'], df['Year']])
#     d = df_filtered_grouped.mean().unstack()

    # Format the data to send as json
    data = {
        "Year": list(df_filtered_grouped.groups.keys()),
        "AverageAge": list(df_filtered_grouped.mean()), 
        "AverageTotal": list(df_grouped.mean())
    }
    
#     for index, row in d.iterrows():
#         data = {
# #             "State": [index],
#             "Year": list(d.columns),
#             "AverageAge": list(row)
#         }

    print (df_filtered_grouped.mean())
    print(data)
#     print(df_ByState.mean())

    return jsonify(data)
    # return data

In [58]:
with app.app_context():
    line('MO')

Year
1995    55.537500
1997    52.525833
1999    55.090909
2001    52.903333
2003    53.618182
2005    54.250909
2007    56.544545
2009    59.094545
2011    56.980000
2013    55.605455
2015    56.936000
2017    58.936000
2019    58.292000
Name: SwornInAge, dtype: float64
{'Year': [1995, 1997, 1999, 2001, 2003, 2005, 2007, 2009, 2011, 2013, 2015, 2017, 2019], 'AverageAge': [55.5375, 52.52583333333334, 55.0909090909091, 52.90333333333333, 53.61818181818182, 54.25090909090909, 56.54454545454546, 59.094545454545454, 56.98, 55.605454545454535, 56.93599999999999, 58.93599999999999, 58.29199999999999], 'AverageTotal': [52.64825539568352, 53.07879999999999, 54.178903107861125, 54.73493670886078, 55.502124542124605, 56.559268738574076, 57.338936936936996, 57.85612788632325, 57.711609403255004, 57.649963963963934, 57.94567518248175, 58.828680926916206, 58.51798148148147]}


In [ ]:
def bar(year):
    stmt = db.session.query(Members_Metadata).statement
    df = pd.read_sql_query(stmt, db.session.bind)
    
    # Filter the data for the year selected
    
    df_filtered = df.loc[df['Year'] == year].reset_index()
        
    # Group the dataframes
    df_filtered_grouped = df_filtered['SwornInAge'].groupby(df_filtered['State'])
    df_grouped = df['SwornInAge'].groupby(df['State'])
    df_ByYear = df['SwornInAge'].groupby([df['Year'], df['State']])
    df_filtered_party = df_filtered['Party'].groupby(df_filtered['State'])
    
    # Get Majority Party for each State

    df_party = df_filtered_party.value_counts().reset_index(name='count')

    df_pivot = df_party.pivot(index='State', columns='Party', values='count')

    df_pivot = df_pivot.fillna(0)

    df_pivot['Majority'] = np.where(df_pivot['D']> df_pivot["R"], 'D', 'R')

    # Format the data to send as json
    data = {
        "State": list(df_filtered_grouped.groups.keys()),
        "AverageAge": list(df_filtered_grouped.mean()), 
        "Party": df_pivot.Majority.values.tolist()
    }

    return jsonify(data)

In [59]:
def bar(year):
    stmt = db.session.query(Members_Metadata).statement
    df = pd.read_sql_query(stmt, db.session.bind)
    
    # Filter the data for the year selected
    
    df_filtered = df.loc[df['Year'] == year].reset_index()
        
    # Group the dataframes
    df_filtered_grouped = df_filtered['SwornInAge'].groupby(df_filtered['State'])
    df_grouped = df['SwornInAge'].groupby(df['State'])
    df_ByYear = df['SwornInAge'].groupby([df['Year'], df['State']])
    df_filtered_party = df_filtered['Party'].groupby(df_filtered['State'])
    
    # Get Majority Party for each State

    df_party = df_filtered_party.value_counts().reset_index(name='count')

    df_pivot = df_party.pivot(index='State', columns='Party', values='count')

    df_pivot = df_pivot.fillna(0)

    df_pivot['Majority'] = np.where(df_pivot['D']> df_pivot["R"], 'D', 'R')

    # Format the data to send as json
    data = {
        "State": list(df_filtered_grouped.groups.keys()),
        "AverageAge": list(df_filtered_grouped.mean()), 
        "Party": df_pivot.Majority.values.tolist()
    }

    return jsonify(data)

In [60]:
with app.app_context():
    bar(2019)

In [62]:
def map(year):
    stmt = db.session.query(Members_Metadata).statement
    df = pd.read_sql_query(stmt, db.session.bind)

    # Filter the data for the states selected

    df_filtered = df.loc[df['Year'] == year].reset_index()

    # Group the dataframes
    df_filtered_grouped_State = df_filtered['SwornInAge'].groupby(df_filtered['State'])

    # Format the data to send as json
    datastate = {
       "State": list(df_filtered_grouped_State.groups.keys()),
       "AverageAge": list(df_filtered_grouped_State.mean())
    }

    return jsonify(datastate)

In [63]:
with app.app_context():
    map(2019)